In [1]:
import tensorflow as tf

import tensorflow.keras as keras
import tensorflow.keras.utils as utils
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
import ta 
import datetime

In [2]:

csvFileAddress = 'csv/FTSE100_M15_202101210130_202206242245.csv'

DJ = pd.read_csv(csvFileAddress,delim_whitespace=True)

DJ['<ISGREEN>'] =  DJ['<CLOSE>'] > DJ['<OPEN>']
DJ['<SIZE>'] =  (DJ['<CLOSE>'] - DJ['<OPEN>']) / DJ['<CLOSE>'] *100
DJ['<VOLATILITY>'] =  DJ['<HIGH>'] - DJ['<LOW>'] /DJ['<CLOSE>']

DJ.drop(['<VOL>'],axis=1,inplace=True)

DJ.fillna(0,inplace=True)

DJ['<EMA30>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=30)
DJ['<EMA50>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=50)
DJ['<EMA200>']= ta.trend.ema_indicator( DJ['<CLOSE>'],window=200)

def Upper(e):
    if e['<ISGREEN>'] : 
        return e['<HIGH>']-e['<CLOSE>']
    return e['<HIGH>']-e['<OPEN>']

def Lower(e):
    if e['<ISGREEN>'] : 
        return e['<OPEN>']-e['<LOW>']
    return e['<CLOSE>']-e['<LOW>']


DJ['<UPPER>'] = DJ.apply(Upper,axis=1)
DJ['<LOWER>'] = DJ.apply(Lower,axis=1)


In [3]:

# Adding TA indicators : 

from ta.volatility import BollingerBands,KeltnerChannel,average_true_range
from ta.trend import MACD

DJ['<RSI>'] = ta.momentum.rsi(DJ['<CLOSE>'],window=15,fillna=1) / 100
# DJ['<CCI>'] = ta.trend.cci(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=14,fillna=1)


keltner = KeltnerChannel(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=50,window_atr=20,multiplier=3,fillna=1)
DJ['<KELTNER_H>'] = keltner.keltner_channel_hband()
DJ['<KELTNER_L>'] = keltner.keltner_channel_lband()
DJ['<KELTNER_M>'] = keltner.keltner_channel_mband()

DJ['<KELT_L_IND>'] = keltner.keltner_channel_lband_indicator()
DJ['<KELT_H_IND>'] = keltner.keltner_channel_hband_indicator()



bollinger =BollingerBands(close=DJ['<CLOSE>'],window=50,window_dev=3,fillna=True)
DJ['<BOL_H_IND>']=bollinger.bollinger_hband_indicator()
DJ['<BOL_L_IND>']=bollinger.bollinger_lband_indicator()

# macd = MACD(DJ['<CLOSE>'],fillna=True)


DJ['<ATR_24>'] = average_true_range(close=DJ['<CLOSE>'],high=DJ['<HIGH>'],low=DJ['<LOW>'],window=30).apply(lambda e : e/100)
DJ['<ATR_24_MULT>'] =abs(DJ['<SIZE>']) / DJ['<ATR_24>'] 


DJ["<ISGREEN>"] = DJ["<ISGREEN>"].astype(int)



In [4]:
timeFrame = (int(DJ.iloc[1][1][1])-int(DJ.iloc[0][1][1]) ) * 60 + (int(DJ.iloc[1][1][3:5])-int(DJ.iloc[0][1][3:5]))
CandlesInDay = 24 * (60//timeFrame)
timeFrame
def candleToTime(j):
    minuteMult = CandlesInDay//24
    k=j//minuteMult
    sth=timeFrame*(j%minuteMult)
    return '{:02d}:{:02d}:00'.format(k,sth)

uniqueDays = DJ.drop_duplicates(subset='<DATE>')
uniqueDays = pd.DataFrame(uniqueDays)

uniqueDaysCount=uniqueDays.shape[0]

newnumparr = np.full((uniqueDaysCount*CandlesInDay,2),'',dtype=np.object_)


for i in range(uniqueDaysCount):
    for j in range(CandlesInDay):
        newnumparr[(i*CandlesInDay)+j]=[uniqueDays.iloc[i][0],candleToTime(j)]

newDF = pd.DataFrame(newnumparr,columns=['<DATE>','<TIME>'])

newestDF = newDF.merge(DJ,on=['<DATE>','<TIME>'],how='left')


In [5]:
newestDF.drop(range(CandlesInDay),inplace=True)
newestDF.fillna(0.0,inplace=True)

In [6]:
newestDF.describe()

,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<TICKVOL>,<SPREAD>,<ISGREEN>,<SIZE>,<VOLATILITY>,<EMA30>,...,<RSI>,<KELTNER_H>,<KELTNER_L>,<KELTNER_M>,<KELT_L_IND>,<KELT_H_IND>,<BOL_H_IND>,<BOL_L_IND>,<ATR_24>,<ATR_24_MULT>
count,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,...,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000,34368.000000
mean,6387.382839,6391.613006,6383.053617,6387.399360,540.142167,8.417190,0.443581,0.000184,6390.721243,6387.198141,...,0.453238,6395.576938,6378.461381,6387.019160,0.276216,0.342615,0.006925,0.009515,0.086695,0.629074
std,2235.021151,2236.485811,2233.522571,2235.026973,551.409403,21.787842,0.496814,0.096138,2236.178418,2234.897897,...,0.189951,2237.811793,2231.873486,2234.839249,0.447132,0.474591,0.082929,0.097079,0.048549,0.671647
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.020394,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,6892.000000,6896.375000,6888.150000,6892.100000,151.000000,1.000000,0.000000,-0.037755,6895.375189,6892.782817,...,0.394352,6900.261500,6884.673000,6892.436167,0.000000,0.000000,0.000000,0.000000,0.062886,0.147300
50%,7111.700000,7115.700000,7106.900000,7111.700000,361.000000,2.000000,0.000000,0.000000,7114.700365,7111.304763,...,0.494371,7118.832000,7101.565667,7110.455000,0.000000,0.000000,0.000000,0.000000,0.082351,0.444962
75%,7341.800000,7346.000000,7336.600000,7341.825000,753.000000,4.000000,1.000000,0.039838,7345.000187,7343.864810,...,0.576584,7352.015167,7335.928167,7344.176667,1.000000,1.000000,0.000000,0.000000,0.109445,0.893992
max,7692.700000,7694.700000,7684.200000,7692.700000,4136.000000,105.000000,1.000000,1.382513,7693.701300,7672.200565,...,0.912481,7676.950667,7654.228000,7665.508667,1.000000,1.000000,1.000000,1.000000,0.340206,8.798593


In [7]:
startCandleHour = 4
startCandleIndex = int(startCandleHour * CandlesInDay )// 24

windowLengthHour = 17
windowLengthCount = int(windowLengthHour * CandlesInDay) // 24

dataLengthHour = 3
dataLengthCount = int(dataLengthHour * CandlesInDay) // 24

# labelStartCandleHour = startCandleHour+windowLengthHour
# labelStartCandleIndex = int(labelStartCandleHour * CandlesInDay) // 24

labelWindowLengthHour = 2
labelWindowLengthCount = int(labelWindowLengthHour * CandlesInDay) // 24

In [ ]:

import threading
import multiprocessing
import logging
import time


takeProfitPoints = 1.002
stopLoss = .998
df = newestDF.drop(['<DATE>','<TIME>'],axis=1)
threadCount = 2

trainNps=[]
labelValues = []

for i in range(threadCount): trainNps.append([])
for i in range(threadCount): labelValues.append([])

maxValues = df.max()

def thread_function(df,threadidx):
    for i in range(0,len(df)-CandlesInDay , CandlesInDay):

        if i% (30*CandlesInDay)==0 : print(i//CandlesInDay)

        currDayDF = df.iloc[i+startCandleIndex : (i+startCandleIndex +windowLengthCount)].copy(deep=1)
    

        for j in range(len(currDayDF)-dataLengthCount-labelWindowLengthCount-2):

            startTrain=j
            endTrain=startTrain+dataLengthCount

            startLabel = endTrain+1
            endLabel = startLabel+labelWindowLengthCount

            trainDF = currDayDF.iloc[startTrain:endTrain].copy(deep=1)
            labelDF=  currDayDF.iloc[startLabel:endLabel].copy(deep=1)

            lowestIndex = labelDF['<LOW>'].idxmin()
            
            lowestPriceInLabels = labelDF['<LOW>'][lowestIndex]

            highesIndex = labelDF['<HIGH>'].idxmax()
            highestPriceInLabels = labelDF['<HIGH>'][highesIndex]

            TP = labelDF['<OPEN>'].iloc[0]*takeProfitPoints
            SL = labelDF['<OPEN>'].iloc[0]*stopLoss

            buySignal = 0

            if highestPriceInLabels < TP and lowestPriceInLabels > SL:
                # idea : halfTP new class in labels
                buySignal=2
                
            elif highestPriceInLabels < TP and lowestPriceInLabels < SL:
                buySignal = 0
            elif highestPriceInLabels > TP and lowestPriceInLabels > SL:
                buySignal = 1
            else :
                tpIndex = 0
                slIndex = 0
                for k in range(len(labelDF)):
                    if labelDF['<HIGH>'].iloc[k] > TP and labelDF['<LOW>'].iloc[k]>SL :
                        tpIndex=k
                    if labelDF['<HIGH>'].iloc[k] < TP and labelDF['<LOW>'].iloc[k]>SL :
                        continue
                    if labelDF['<HIGH>'].iloc[k] < TP and labelDF['<LOW>'].iloc[k]<SL :
                        slIndex=k
                buySignal = int(tpIndex<slIndex)
                if slIndex==tpIndex:
                    buySignal = 2
            

            # trainDF =  trainDF /df.abs().max()
            trainDF=(trainDF-df.min())/(df.max()-df.min())

            # trainDF = trainDF.drop(['<OPEN>','<ATR_24>','<EMA30>','<RSI>','<CCI>','<KELTNER_M>','<KELTNER_L>','<KELTNER_H>','<GREEN>'],axis=1)
            trainDF = trainDF[['<ATR_24_MULT>','<TICKVOL>','<KELT_L_IND>','<KELT_H_IND>','<BOL_L_IND>','<BOL_H_IND>']]


            trainNp =(trainDF.to_numpy(dtype=np.float32))
            # trainNp = np.rot90( trainDF.to_numpy())


            labelValues[threadidx].append(buySignal)
            trainNps[threadidx].append(trainNp)

lenDFthread =  len(df) / threadCount
threads = []

for index in range(threadCount):
    x = multiprocessing.Process(target=thread_function, args=(df.iloc[int(index*lenDFthread):int((1+index)*lenDFthread-1)],index))
    x.start()
    threads.append(x)
for index,thread in enumerate(threads):
    thread.join()




In [14]:
datas=[]
labs=[]
for i in range(threadCount): 
    datas = datas+trainNps[i]
    labs = labs + labelValues[i]


In [44]:
len(datas),len(labs)

(16376, 16376)

In [15]:

l = len(datas)
windows2= np.array(datas)

labs = np.array(labs)


train_data = windows2[:int(l*.75)]
val_data = windows2[int(l*.75)+1:]



train_labels = labs[:int(l*.75)].astype(np.uint8)
val_labels = labs[int(l*.75)+1:].astype(np.uint8)


len(train_data),len(train_labels)


(12282, 12282)

In [38]:
len(val_data),len(val_labels)

(4105, 4105)

In [16]:
pd.DataFrame(labs,columns=['signal']).value_counts()/len(labs)

signal
0         0.39222
1         0.38361
2         0.22417
dtype: float64

In [47]:
tf.random.set_seed(42)

from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D,LSTM, ConvLSTM1D,Flatten,SimpleRNN,GRU,AveragePooling2D

tf.random.set_seed(42)

model_rnn = keras.Sequential([
    LSTM(32,input_shape=train_data[0].shape,dropout=.1,return_sequences=1),
    LSTM(64,dropout=.1,return_sequences=1),
    LSTM(128,dropout=.1,return_sequences=0),
    # Flatten(),
    Dense(100,activation='relu'),
    Dense(3,activation='sigmoid'),

])

model_rnn.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(),
    optimizer = 'adam',
    metrics=['accuracy']
)

model_rnn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 12, 32)            4992      
                                                                 
 lstm_1 (LSTM)               (None, 12, 64)            24832     
                                                                 
 lstm_2 (LSTM)               (None, 128)               98816     
                                                                 
 dense (Dense)               (None, 100)               12900     
                                                                 
 dense_1 (Dense)             (None, 3)                 303       
                                                                 
Total params: 141,843
Trainable params: 141,843
Non-trainable params: 0
_________________________________________________________________


In [48]:
model_rnn.fit(train_data,train_labels,validation_data=(val_data,val_labels),batch_size=64,epochs=200,verbose=2)

Epoch 1/200
192/192 - 9s - loss: 1.0855 - accuracy: 0.3750 - val_loss: 1.0291 - val_accuracy: 0.4645 - 9s/epoch - 47ms/step
Epoch 2/200
192/192 - 5s - loss: 1.0825 - accuracy: 0.3734 - val_loss: 1.0272 - val_accuracy: 0.4517 - 5s/epoch - 26ms/step
Epoch 3/200
192/192 - 5s - loss: 1.0802 - accuracy: 0.3772 - val_loss: 1.0451 - val_accuracy: 0.4495 - 5s/epoch - 26ms/step
Epoch 4/200
192/192 - 5s - loss: 1.0774 - accuracy: 0.3810 - val_loss: 1.0317 - val_accuracy: 0.4491 - 5s/epoch - 26ms/step
Epoch 5/200
192/192 - 5s - loss: 1.0761 - accuracy: 0.3860 - val_loss: 1.0226 - val_accuracy: 0.4439 - 5s/epoch - 25ms/step
Epoch 6/200
192/192 - 5s - loss: 1.0716 - accuracy: 0.3882 - val_loss: 1.0341 - val_accuracy: 0.4586 - 5s/epoch - 26ms/step
Epoch 7/200
192/192 - 5s - loss: 1.0691 - accuracy: 0.3835 - val_loss: 1.0346 - val_accuracy: 0.4327 - 5s/epoch - 26ms/step
Epoch 8/200
192/192 - 5s - loss: 1.0677 - accuracy: 0.3884 - val_loss: 1.0275 - val_accuracy: 0.4237 - 5s/epoch - 26ms/step
Epoch 9/

KeyboardInterrupt: 